### **0. Set-up**

In [29]:
# Import libraries and utils
%run '../../utils.ipynb'
%run '../../utils_structured.py'

In [2]:
# Get api key
load_dotenv()
OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY')

# Set client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=OPENROUTER_API_KEY,
)

In [3]:
# Load dataframe
en_simlex = pd.read_csv("../../../data/dataset/cleaned-en-simlex-999.csv")

# Select subset
en_simlex = en_simlex.iloc[333:666]

# Convert to tuple
tuples_list = list(zip(en_simlex['word1'], en_simlex['word2']))

In [4]:
# Show results
tuples_list

[('game', 'fun'),
 ('weekend', 'week'),
 ('couple', 'pair'),
 ('woman', 'wife'),
 ('sheep', 'cattle'),
 ('purse', 'bag'),
 ('ceiling', 'cathedral'),
 ('bean', 'coffee'),
 ('wood', 'paper'),
 ('top', 'side'),
 ('crime', 'fraud'),
 ('pain', 'harm'),
 ('lover', 'companion'),
 ('evening', 'dusk'),
 ('father', 'daughter'),
 ('wine', 'liquor'),
 ('cow', 'goat'),
 ('belief', 'opinion'),
 ('reality', 'illusion'),
 ('pact', 'agreement'),
 ('wealth', 'poverty'),
 ('accident', 'emergency'),
 ('battle', 'conquest'),
 ('friend', 'teacher'),
 ('illness', 'infection'),
 ('game', 'trick'),
 ('brother', 'son'),
 ('aunt', 'nephew'),
 ('worker', 'mechanic'),
 ('doctor', 'orthodontist'),
 ('oak', 'maple'),
 ('bee', 'queen'),
 ('car', 'bicycle'),
 ('goal', 'quest'),
 ('august', 'month'),
 ('army', 'squad'),
 ('cloud', 'weather'),
 ('physician', 'doctor'),
 ('canyon', 'valley'),
 ('river', 'valley'),
 ('sun', 'sky'),
 ('target', 'arrow'),
 ('chocolate', 'pie'),
 ('circumstance', 'situation'),
 ('opinion', '

### **1. Define and Evaluate Parameters**

In [5]:
prompt = ("Rate the semantic similarity of the word pair: [('{word1}'), ('{word2}')] on a scale from 0 to 10, "
          "where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. "
          "Use two decimals. Do not provide additional explanations or context.")

In [6]:
# Define model
model = "mistralai/mistral-7b-instruct-v0.3"

# Set sample size
sample_size = 15

# Delay between individual API calls
delay = 0.01

# Define number of sublists
n_sublists = 333

In [7]:
# Split list
chunks = split_into_n_lists(tuples_list, n_sublists)

# Count the number of lists
print(len(chunks))

333


In [8]:
# Print prompts for each chunk
print_prompts_single(chunks, sample_size, prompt)

Rate the semantic similarity of the word pair: [('game'), ('fun')] on a scale from 0 to 10, where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. Use two decimals. Do not provide additional explanations or context.
Rate the semantic similarity of the word pair: [('game'), ('fun')] on a scale from 0 to 10, where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. Use two decimals. Do not provide additional explanations or context.
Rate the semantic similarity of the word pair: [('game'), ('fun')] on a scale from 0 to 10, where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. Use two decimals. Do not provide additional explanations or context.
Rate the semantic similarity of the word pair: [('game'), ('fun')] on a scale from 0 to 10, where 0 represents no semantic similarity, and 10 represents perfect semantic similarity. Use two decimals. Do not provide additional explanations or conte

In [9]:
# Load encoding
encoding = tiktoken.get_encoding("cl100k_base")

# Count tokens per chunk
token_counts = count_tokens_with_tiktoken_single(chunks, prompt)

# Show results
print("Token counts for each formatted prompt:", token_counts)

Token counts for each formatted prompt: [53, 54, 54, 53, 55, 54, 56, 53, 53, 53, 54, 54, 54, 55, 53, 54, 54, 54, 54, 55, 54, 54, 54, 53, 55, 54, 54, 56, 55, 55, 55, 53, 54, 53, 54, 55, 53, 54, 56, 54, 53, 53, 54, 56, 54, 55, 55, 54, 55, 54, 54, 54, 55, 53, 54, 54, 54, 53, 53, 54, 53, 55, 53, 54, 54, 55, 54, 53, 54, 54, 54, 53, 54, 55, 54, 53, 54, 53, 53, 56, 53, 55, 53, 53, 54, 55, 54, 55, 55, 53, 54, 54, 56, 53, 54, 55, 54, 53, 54, 54, 55, 53, 55, 55, 54, 53, 55, 54, 55, 56, 56, 54, 54, 55, 54, 53, 54, 54, 54, 54, 55, 55, 54, 55, 54, 53, 54, 55, 54, 53, 54, 53, 54, 54, 53, 54, 54, 54, 53, 55, 53, 55, 53, 54, 53, 54, 53, 54, 53, 53, 53, 53, 53, 56, 54, 54, 54, 54, 54, 54, 53, 53, 53, 54, 54, 54, 55, 54, 55, 53, 53, 56, 54, 53, 53, 54, 53, 53, 54, 53, 53, 54, 55, 53, 55, 55, 54, 54, 54, 54, 56, 53, 55, 53, 54, 55, 54, 53, 54, 54, 54, 54, 54, 54, 54, 53, 55, 53, 53, 54, 53, 54, 53, 53, 54, 53, 54, 54, 53, 53, 53, 54, 54, 53, 54, 54, 53, 55, 55, 53, 54, 55, 55, 55, 54, 53, 54, 55, 54, 53,

In [10]:
# Max RPD = 10.000
len(token_counts*15)

4995

In [11]:
# Number of total tokens
print(sum(token_counts))

17975


### **2. Extract Data**

In [12]:
# Get results from API
response = get_responses_single_structured(prompt, chunks, model, sample_size, delay, client)

Processing: 100%|██████████| 4995/4995 [1:49:12<00:00,  1.31s/chunk]  

Total time taken: 6552.30 seconds


In [13]:
# Define filepath
file_path = '../../../data/mistral-7b-instruct-v0.3/response/en/f9-2.json'

# Check if file already exists
if not os.path.exists(file_path):
    with open(file_path, 'w') as f:
        json.dump(response, f)
        print("File saved successfully.")
else:
    print("File already exists. JSON was not saved to prevent overwriting.")

File saved successfully.


### **3. Process and Inspect Data**

In [32]:
# Process data into dictionary
data_dict = process_responses_structured(response)

# Check for values higher/lower then sample size
higher_lower_samples = {key: value for key, value in data_dict.items() if len(value) < sample_size or len(value) > sample_size}

# Show results
print(higher_lower_samples)

{}


In [33]:
# Print duplicate word pairs
print_duplicate_word_pairs(en_simlex, data_dict)

Empty DataFrame
Columns: [Combined_Columns]
Index: []
Empty DataFrame
Columns: [Combined_Columns]
Index: []


In [34]:
# Convert dict to Pandas DataFrame
df = create_dataframe(data_dict)

# Show results
df

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15
0,game,fun,8.50,7.50,8.50,7.50,7.89,7.67,7.89,7.68,7.67,8.50,7.65,9.50,7.50,8.25,7.88
1,weekend,week,8.67,8.50,8.50,8.67,8.67,8.50,8.50,8.50,8.50,8.50,7.50,8.50,8.50,8.50,8.50
2,couple,pair,9.80,9.80,9.20,9.80,9.80,9.80,9.80,9.80,9.80,9.80,9.80,9.80,9.80,9.50,9.50
3,woman,wife,8.75,8.50,8.50,8.50,8.50,8.50,8.75,8.67,8.50,8.50,8.50,7.88,8.50,8.50,7.80
4,sheep,cattle,7.25,7.68,7.67,6.50,7.25,7.65,7.50,7.50,7.50,7.68,7.67,7.22,7.50,7.25,7.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,letter,paragraph,6.67,7.65,6.67,7.25,7.67,6.66,6.50,7.25,6.45,7.68,1.00,7.22,7.25,7.25,7.68
329,page,paragraph,8.50,8.67,7.89,8.50,7.50,7.67,8.25,8.75,8.20,7.50,7.67,8.50,8.67,8.50,7.80
330,steeple,chapel,7.90,7.50,7.50,8.50,7.68,7.50,7.65,7.63,7.80,7.22,7.25,7.89,7.68,7.65,7.68
331,muscle,bone,2.00,7.25,8.50,7.89,7.35,8.50,7.50,2.00,8.67,7.67,2.00,7.50,7.68,7.22,7.45


In [35]:
# Count null values
count_null_values = df.isnull().sum()

# Show results
print("Null value counts per column:", count_null_values)

Null value counts per column: word1                  0
word2                  0
similarity_score_1     0
similarity_score_2     0
similarity_score_3     0
similarity_score_4     0
similarity_score_5     0
similarity_score_6     0
similarity_score_7     0
similarity_score_8     0
similarity_score_9     0
similarity_score_10    0
similarity_score_11    0
similarity_score_12    0
similarity_score_13    0
similarity_score_14    0
similarity_score_15    0
dtype: int64


In [36]:
# Check for rows with at least one null value
rows_with_null = df[df.isnull().any(axis=1)]

# Show results
rows_with_null

,word1,word2,similarity_score_1,similarity_score_2,similarity_score_3,similarity_score_4,similarity_score_5,similarity_score_6,similarity_score_7,similarity_score_8,similarity_score_9,similarity_score_10,similarity_score_11,similarity_score_12,similarity_score_13,similarity_score_14,similarity_score_15


### **4. Fix Faulty Word Pairs**

In [37]:
# ...

### **5. Export Data**

In [38]:
# Define file_path
file_path = '../../../data/mistral-7b-instruct-v0.3/processed/en/f9-2.csv'

# Check if file already exists
if not os.path.exists(file_path):
    df.to_csv(file_path, index=False)
    print("File saved successfully.")
else:
    print("File already exists. Dataframe was not saved to prevent overwriting.")

File saved successfully.
